In [1]:
import torch
import torchvision.models as models
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from pathlib import Path
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch import optim
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from skimage.color import rgba2rgb

In [2]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

with open('data.zip', 'wb') as f:
    s3.download_fileobj('intelinair-data-releases', 'longitudinal-nutrient-deficiency/Longitudinal_Nutrient_Deficiency.zip', f)

In [ ]:
unzip -qq ./data.zip

In [2]:
%load_ext tensorboard

In [8]:
from torch.utils.tensorboard import SummaryWriter # TensorBoard support

tb = SummaryWriter(comment='myboard')

# Device configuration
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

device

device(type='cpu')

## Готовим датасет

Скачиваем данные, передаем трансформер (предобработку), визуализируем  примеры семплов, создаем dataloaders для обучения и тестирования модели

In [9]:
from torch.utils.data import Dataset

import cv2
import PIL
class CustomDataset(Dataset):
    def __init__(self, img_dir, transform=None):
     #   self.img_labels = os.walk(self.img_dir)][1:]
        self.img_labels = [x[0]+"/nutrient_mask_g0.png" for x in os.walk(datasetDir)][1:]
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
     #   lst = [x[0] for x in os.walk(datasetDir)][1:][]
      #  folder_list = [x[0] for x in os.walk(self.img_dir)][1:]
#pil_image = PIL.Image.open('Image.jpg').convert('RGB') 

        image = PIL.Image.open(os.path.join(datasetDir, "field_"+"{:03d}".format(idx+1)+"/image_i0.png")).convert('RGB') #убираем ф
        label =  PIL.Image.open(os.path.join(datasetDir, "field_"+"{:03d}".format(idx+1)+"/nutrient_mask_g0.png")).convert('1')
   #     image = rgba2rgb(PIL.Image.open(os.path.join(datasetDir, "field_"+"{:03d}".format(idx+1)+"/image_i0.png")))
    #    label =  rgba2rgb(PIL.Image.open(os.path.join(datasetDir, "field_"+"{:03d}".format(idx+1)+"/nutrient_mask_g0.png")))


        if self.transform:
            image = self.transform(image)#[:,:,:3]
            label = self.transform(label)#[:,:,:3]

        return image, label

In [10]:
import os
datasetDir = os.path.join(os.getcwd(),'Longitudinal_Nutrient_Deficiency')
mydataset = CustomDataset(img_dir = datasetDir, transform = transforms.Compose([transforms.Resize(480), transforms.CenterCrop(480),transforms.ToTensor()]))


In [11]:
from sklearn.model_selection import train_test_split

training_data, testing_data = train_test_split(mydataset, test_size=0.2, random_state=25)


In [12]:
train_loader =  torch.utils.data.DataLoader(training_data, 
                                          batch_size=40, 
                                          shuffle=True, 
                                          num_workers=1)
    
test_loader = torch.utils.data.DataLoader(testing_data, 
                                          batch_size=40, 
                                          shuffle=True, 
                                          num_workers=1)


In [13]:
xb, yb = next(iter(train_loader))
xb.shape, yb.shape

(torch.Size([40, 3, 480, 480]), torch.Size([40, 1, 480, 480]))

In [14]:
from torch import nn
class UNET(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = self.contract_block(in_channels, 32, 7, 3)
        self.conv2 = self.contract_block(32, 64, 3, 1)
        self.conv3 = self.contract_block(64, 128, 3, 1)

        self.upconv3 = self.expand_block(128, 64, 3, 1)
        self.upconv2 = self.expand_block(64*2, 32, 3, 1)
        self.upconv1 = self.expand_block(32*2, out_channels, 3, 1)

    def __call__(self, x):

        # downsampling part
        conv1 = self.conv1(x)
        conv2 = self.conv2(conv1)
        conv3 = self.conv3(conv2)

        upconv3 = self.upconv3(conv3)

        upconv2 = self.upconv2(torch.cat([upconv3, conv2], 1))
        upconv1 = self.upconv1(torch.cat([upconv2, conv1], 1))

        return upconv1

    def contract_block(self, in_channels, out_channels, kernel_size, padding):

        contract = nn.Sequential(
            torch.nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=padding),
            torch.nn.BatchNorm2d(out_channels),
            torch.nn.ReLU(),
            torch.nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, stride=1, padding=padding),
            torch.nn.BatchNorm2d(out_channels),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
                                 )

        return contract

    def expand_block(self, in_channels, out_channels, kernel_size, padding):

        expand = nn.Sequential(torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=padding),
                            torch.nn.BatchNorm2d(out_channels),
                            torch.nn.ReLU(),
                            torch.nn.Conv2d(out_channels, out_channels, kernel_size, stride=1, padding=padding),
                            torch.nn.BatchNorm2d(out_channels),
                            torch.nn.ReLU(),
                            torch.nn.ConvTranspose2d(out_channels, out_channels, kernel_size=3, stride=2, padding=1, output_padding=1) 
                            )
        return expand

In [15]:
#PyTorch
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice

In [16]:
unet = UNET(3,1)

In [17]:
import time
def train(model, train_dl, valid_dl, loss_fn, optimizer, acc_fn, epochs=1):
    start = time.time()
    model.cpu()

    train_loss, valid_loss = [], []

    best_acc = 0.0

    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch, epochs - 1))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train(True)  # Set trainind mode = true
                dataloader = train_dl
            else:
                model.train(False)  # Set model to evaluate mode
                dataloader = valid_dl

            running_loss = 0.0
            running_acc = 0.0

            step = 0

            # iterate over data
            for x, y in dataloader:
                x = x.cpu()
                y = y.cpu()
                step += 1

                # forward pass
                if phase == 'train':
                    # zero the gradients
                    optimizer.zero_grad()
                    outputs = model(x)
                    from IPython.core.debugger import Tracer
                  #  Tracer()() # на loss валится
                    
                    loss = loss_fn(outputs, y)
                    

                    # the backward pass frees the graph memory, so there is no 
                    # need for torch.no_grad in this training pass
                    loss.backward()
                    optimizer.step()
                    # scheduler.step()

                else:
                    with torch.no_grad():
                        outputs = model(x)
                        loss = loss_fn(outputs, y.long())

                # stats - whatever is the phase
                acc = acc_fn(outputs, y)

                running_acc  += acc*dataloader.batch_size
                running_loss += loss*dataloader.batch_size 

              #  if step % 10 == 0:
                    # clear_output(wait=True)
           #         print('Current step: {}  Loss: {}  Acc: {}  AllocMem (Mb): {}'.format(step, loss, acc, torch.cuda.memory_allocated()/1024/1024))
                    # print(torch.cuda.memory_summary())

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_acc / len(dataloader.dataset)

            print('{} Loss: {:.4f} Acc: {}'.format(phase, epoch_loss, epoch_acc))

            train_loss.append(epoch_loss) if phase=='train' else valid_loss.append(epoch_loss)

    time_elapsed = time.time() - start
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    
    
    return train_loss, valid_loss    

def acc_metric(predb, yb):
    return (predb.argmax(dim=1) == yb.cpu()).float().mean()

In [18]:
loss_fn = DiceLoss()
opt = torch.optim.Adam(unet.parameters(),lr=0.01)
train_loss, valid_loss = train(unet,train_loader,test_loader,loss_fn,opt,acc_metric,epochs=5)

Epoch 0/4
----------


/Users/jetpacula/miniforge3/envs/firstEnv/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


train Loss: 0.9540 Acc: 0.9936091303825378
valid Loss: 0.9363 Acc: 0.975995659828186
Epoch 1/4
----------
train Loss: 0.9447 Acc: 0.9930424094200134
valid Loss: 0.9339 Acc: 0.9762735366821289
Epoch 2/4
----------
train Loss: 0.9370 Acc: 0.9935832023620605
valid Loss: 0.9375 Acc: 0.9761801362037659
Epoch 3/4
----------
train Loss: 0.9244 Acc: 0.9935427308082581
valid Loss: 0.9030 Acc: 0.9760116338729858
Epoch 4/4
----------
train Loss: 0.9182 Acc: 0.993461012840271
valid Loss: 0.9057 Acc: 0.9759831428527832
Training complete in 50m 41s


tb.close()
%tensorboard --logdir=runs